# Preprocessing
To clean `JSON` weather and taxi avail files called from NEA and LTA sites <br>

## Libraries

In [1]:
import numpy as np
import pandas as pd
import gcsfs


from scipy.spatial import distance
from datetime import date, timedelta

import math
from collections import Counter


from tqdm import tqdm
from datetime import datetime

import geopandas as gpd

from src import assignment

import warnings
warnings.filterwarnings('ignore')

## 2. Processing weather and taxi tabular files 
Merge the data together into 1 dataframe<br>
This is only for ONE timestamp. Timestamps should be cleaned min is either 00 or 30; sec is 00<br>
Eg 2019-01-01T12:30:00 <br>

Weather files: <br>
1. rainfall w stations value     ---   timestamp | station_id | value
2. rainfall w stations latlon    ---  timestamp | station_id | latitude | longitude
3. humidity w stations value     ---   timestamp | station_id | value
4. humidity w stations latlon    ---   timestamp | station_id | latitude | longitude
5. temperature w stations value  ---  timestamp | station_id | value
6. temperature w stations latlon --- timestamp | station_id | latitude | longitude
<br>

Taxi files: <br>
7. taxi avail json file

In [105]:
project = 'ml-eng-cs611-group-project'
nea_bucket = 'ml-eng-cs611-group-project-nea'
taxi_bucket = 'ml-eng-cs611-group-project-taxis'
dataset_id='taxi_dataset_reference'
table_id_nea='assignment-station-v2'
measure = 'rainfall'
measures = ['rainfall','air-temperature','relative-humidity']

fs = gcsfs.GCSFileSystem(project=project)
nea_filenames = fs.glob('/'.join([nea_bucket,measure,"*"]))

### Read grid file

In [3]:
def get_grid_data(gridfile:str):
    '''Read the shpfile containing Singpaore grid data
    Args:
        gridfile:str:Path to gridfile
    Returns
        geopandas.DataFrame with following schema
        - grid_num 
        - intersect
        - geometry (active geometry)        
        - latlon (tuple)
    '''
    grids = gpd.read_file(gridfile)
    grids['centroid'] = grids['geometry'].apply(lambda x: x.centroid) # get grids' centroid

    # convert to dataframe
    grids_df = pd.DataFrame(grids)
    grids_df['centroid'] = grids_df['centroid'].astype(str)
    grids_df['latlon'] = grids_df['centroid'].apply(lambda x: (float(x.split(' ')[1][1:]), float(x.split(' ')[2][:-1])))
    grids_df['grid_num']=grids_df['grid_num'].astype(int)
    return grids_df

In [4]:
grids_file = './updated codes/filter_grids_2/filter_grids_2.shp' ## To change directory

grids_df = get_grid_data(grids_file)

# Get unique grid_num
grid_nums = list(grids_df['grid_num'].unique())

In [5]:
grids_df

,grid_num,intersect,geometry,centroid,latlon
0,9,1,"POLYGON ((103.76364 1.47500, 103.78409 1.47500...",POINT (103.77386363636363 1.4647307692307692),"(103.77386363636363, 1.4647307692307692)"
1,10,1,"POLYGON ((103.78409 1.47500, 103.80455 1.47500...",POINT (103.79431818181817 1.464730769230769),"(103.79431818181817, 1.464730769230769)"
2,11,1,"POLYGON ((103.80455 1.47500, 103.82500 1.47500...",POINT (103.8147727272727 1.464730769230769),"(103.8147727272727, 1.464730769230769)"
3,12,1,"POLYGON ((103.82500 1.47500, 103.84545 1.47500...",POINT (103.83522727272724 1.464730769230769),"(103.83522727272724, 1.464730769230769)"
4,13,1,"POLYGON ((103.84545 1.47500, 103.86591 1.47500...",POINT (103.85568181818182 1.464730769230769),"(103.85568181818182, 1.464730769230769)"
...,...,...,...,...,...
176,255,1,"POLYGON ((103.84545 1.24908, 103.86591 1.24908...",POINT (103.85568181818182 1.2388076923076925),"(103.85568181818182, 1.2388076923076925)"
177,265,1,"POLYGON ((103.60000 1.22854, 103.62045 1.22854...",POINT (103.61022727272726 1.218269230769231),"(103.61022727272726, 1.218269230769231)"
178,266,1,"POLYGON ((103.62045 1.22854, 103.64091 1.22854...",POINT (103.6306818181818 1.2182692307692313),"(103.6306818181818, 1.2182692307692313)"
179,267,1,"POLYGON ((103.64091 1.22854, 103.66136 1.22854...",POINT (103.65113636363635 1.218269230769231),"(103.65113636363635, 1.218269230769231)"


### Load weather datasets

In [6]:
def query_nea_metadata(measure:str, query:str, project_id='ml-eng-cs611-group-project',dataset_id='taxi_dataset_reference'):
    '''Query NEA BigQuery for metadata
    Args:
        measure:            rainfall, relative-humidity or air-temperature
        query_timestamp:    i.e. 2022-06-01 13:15:00
        project_id:         Google Cloud project_id
        dataset_id:         Google Cloud dataset_id
    Returns:
        pandas.DataFrame containing metadata for selected measure
    '''
    table_dict={'rainfall':'rainfall-metadata','relative-humidity':'relative-humidity-metadata','air-temperature':'air-temperature-metadata'}

    sql = f"""
    SELECT timestamp, station, latitude, longitude
    FROM `{dataset_id}.{table_dict[measure]}`
    WHERE timestamp = '{query}'
    """

    return pd.read_gbq(sql, project_id=project_id)


def query_nea_items(measure:str, query:str, project_id='ml-eng-cs611-group-project',dataset_id='taxi_dataset_reference'):
    '''Query NEA BigQuery for metadata
    Args:
        measure:            rainfall, relative-humidity or air-temperature
        query_timestamp:    i.e. 2022-06-01 13:15:00
        project_id:         Google Cloud project_id
        dataset_id:         Google Cloud dataset_id
    Returns:
        pandas.DataFrame containing metadata for selected measure
    '''
    table_dict={'rainfall':'rainfall-items','relative-humidity':'relative-humidity-items','air-temperature':'air-temperature-items'}

    sql = f"""
    SELECT timestamp, station_id, value
    FROM `{dataset_id}.{table_dict[measure]}`
    WHERE timestamp = '{query}'
    """

    return pd.read_gbq(sql, project_id=project_id)


def query_nea_view(measure:str, query:str, project_id='ml-eng-cs611-group-project',dataset_id='taxi_dataset_views'):
    '''Query NEA BigQuery for metadata
    Args:
        measure:            rainfall, relative-humidity or air-temperature
        query_timestamp:    i.e. 2022-06-01 13:15:00
        project_id:         Google Cloud project_id
        dataset_id:         Google Cloud dataset_id
    Returns:
        pandas.DataFrame containing metadata for selected measure
    '''
    table_dict={'rainfall':'view-rainfall','relative-humidity':'view-relative-humidity','air-temperature':'view-air-temperature'}

    sql = f"""
    SELECT *
    FROM `{dataset_id}.{table_dict[measure]}`
    WHERE timestamp = '{query}'
    """

    return pd.read_gbq(sql, project_id=project_id)


def query_taxi_availability(query_timestamp:str, project_id='ml-eng-cs611-group-project',dataset_id='taxi_dataset_reference'):
    '''Query LTA BigQuery for taxi coordinates
    Args:        
        query_timestamp:    i.e. 2022-06-01 13:15:00
        project_id:         Google Cloud project_id
        dataset_id:         Google Cloud dataset_id
    Returns:
        pandas.DataFrame containing metadata for taxi data
    '''
    sql = f"""
    SELECT *
    FROM `{dataset_id}.taxi-availability`
    WHERE timestamp = '{query_timestamp}'
    """

    return pd.read_gbq(sql, project_id=project_id)

In [97]:
def assign_measure(query,measure,grids_df):

    df_metadata=query_nea_metadata(measure=measure,query=query)
    df_metadata['latlon']=df_metadata[['longitude','latitude',]].apply(tuple,axis=1)
    df_metadata.index=df_metadata['station']
    grid_nums = list(grids_df['grid_num'].unique())    
    assignment = {}

    for i in tqdm(range(len(grid_nums)),desc='Grids assigned'):
        grid_coordinates = grids_df.iloc[i]['latlon'] # latlon of row i grid_num    
        distances = df_metadata['latlon'].apply(lambda x: distance.euclidean(x,grid_coordinates))
        distance_sorted = distances.sort_values()
        ranked={k:v for k,v in enumerate(distance_sorted.index.values)}
        assignment[grids_df.iloc[i]['grid_num']]=ranked
        
    measure_df=pd.DataFrame(assignment).T.reset_index().melt(id_vars='index')
    measure_df.rename(columns={'index':'grid_num','variable':'rank','value':'station_id'},inplace=True)
    measure_df['rank']=measure_df['rank'].apply(int)    
    measure_df['date_active']=pd.to_datetime(query)
    measure_df['date_inactive']=pd.to_datetime('2050-12-31 00:00:00')
    measure_df = measure_df[['grid_num','date_active','date_inactive','rank','station_id']]
    return measure_df

In [99]:
def assign_grids_v2(grids_df, query):
    '''
    Arguments:            
        grids_df:geopandas.DataFrame    DataFrame containing Singapore map data
        query:str                       Timestamp for mapping
    Yields:

    '''
    measures=['air-temperature','rainfall','relative-humidity']    
    assignment = {}
    for measure in measures:
        print(f"Assigning stations for [{measure}]")
        result=assign_measure(query,measure,grids_df)
        assignment[measure]=result

    assignment_df=pd.concat(assignment).reset_index()
    assignment_df.drop(['level_0','level_1'],axis=1,inplace=True)
    
    return assignment_df

In [102]:
query = '2022-06-01 13:15:00'
assignment_df = assign_grids_v2(grids_df,query)
assignment_df

Assigning stations for [air-temperature]


Grids assigned: 100%|██████████| 181/181 [00:00<00:00, 1450.63it/s]


Assigning stations for [rainfall]


Grids assigned: 100%|██████████| 181/181 [00:00<00:00, 676.16it/s]


Assigning stations for [relative-humidity]


Grids assigned: 100%|██████████| 181/181 [00:00<00:00, 1543.41it/s]


,grid_num,date_active,date_inactive,rank,station_id,measure
0,9,2022-06-01 13:15:00,2050-12-31,0,S100,air-temperature
1,10,2022-06-01 13:15:00,2050-12-31,0,S100,air-temperature
2,11,2022-06-01 13:15:00,2050-12-31,0,S100,air-temperature
3,12,2022-06-01 13:15:00,2050-12-31,0,S100,air-temperature
4,13,2022-06-01 13:15:00,2050-12-31,0,S100,air-temperature
...,...,...,...,...,...,...
14113,255,2022-06-01 13:15:00,2050-12-31,4,S106,relative-humidity
14114,265,2022-06-01 13:15:00,2050-12-31,4,S106,relative-humidity
14115,266,2022-06-01 13:15:00,2050-12-31,4,S106,relative-humidity
14116,267,2022-06-01 13:15:00,2050-12-31,4,S106,relative-humidity


In [106]:
print("Uploading results to Bigquery...")
assignment_df.to_gbq('.'.join([dataset_id,table_id_nea]),project,if_exists='append')

Uploading results to Bigquery...


100%|██████████| 1/1 [00:00<00:00, 10010.27it/s]


In [63]:
from datetime import timedelta
date=datetime.strptime('2022-06-01 13:15:00','%Y-%m-%d %H:%M:%S')
date_list=[date + timedelta(minutes=15*x) for x in range(1)]
date_list

[datetime.datetime(2022, 6, 1, 13, 15)]

### Processing weather files

### Load taxi json file

In [29]:
taxi_query_timestamp = '2022-06-01 13:15:00'
taxi = query_taxi_availability(taxi_query_timestamp)

In [30]:
taxi

,timestamp,longitude,latitude
0,2022-06-01 13:15:00+00:00,103.825820,1.250000
1,2022-06-01 13:15:00+00:00,103.940568,1.351660
2,2022-06-01 13:15:00+00:00,103.724660,1.335960
3,2022-06-01 13:15:00+00:00,103.822530,1.304710
4,2022-06-01 13:15:00+00:00,103.851526,1.301081
...,...,...,...
2376,2022-06-01 13:15:00+00:00,103.885470,1.310000
2377,2022-06-01 13:15:00+00:00,103.887910,1.310000
2378,2022-06-01 13:15:00+00:00,103.889060,1.310000
2379,2022-06-01 13:15:00+00:00,103.896360,1.310000


### Processing taxi file

In [50]:

def assign_taxis(taxi_df,grids):
    def get_grid_longitude(longitude:float):
        return math.ceil((longitude-103.6)/0.020454545454545583)

    def get_grid_latitude(latitude:float):
        return (13 - math.ceil((latitude -1.208)/0.020538461538461547))*22
    longitude_array = taxi_df['longitude'].to_numpy()
    latitude_array = taxi_df['latitude'].to_numpy()

    test = [get_grid_longitude(longitude_array[i]) + get_grid_latitude(latitude_array[i]) for i in range(len(longitude_array))]

    # getting dictionary of items
    c = Counter(test)

    # Getting taxi_count for relevant grid_num
    df_taxicount = pd.DataFrame({'grid_num': [float(x) for x in list(c.keys())], 
                                 'taxi_count': [x[1] for x in list(c.items())]})

    # Get full list of grid_num as a dataframe:  grid_num | timestamp
    all_grids = grids[['grid_num']]
    all_grids['timestamp'] = taxi_df['timestamp'][0]

    # Merge all_grids and df_taxicount
    taxi_clean = pd.merge(all_grids, df_taxicount, how='left')
    taxi_clean['taxi_count'] = taxi_clean['taxi_count'].fillna(0) #fill missing taxi_count = 0
    taxi_clean['grid_num']=taxi_clean['grid_num'].apply(int)    
    return taxi_clean

In [51]:
taxi_df=assign_taxis(taxi,grids_df)
taxi_df

,grid_num,timestamp,taxi_count
0,9,2022-06-01 13:15:00+00:00,0.0
1,10,2022-06-01 13:15:00+00:00,0.0
2,11,2022-06-01 13:15:00+00:00,1.0
3,12,2022-06-01 13:15:00+00:00,0.0
4,13,2022-06-01 13:15:00+00:00,0.0
...,...,...,...
176,255,2022-06-01 13:15:00+00:00,0.0
177,265,2022-06-01 13:15:00+00:00,0.0
178,266,2022-06-01 13:15:00+00:00,0.0
179,267,2022-06-01 13:15:00+00:00,0.0


## 3. Merging all the cleaned files

In [34]:
## Merge all together
merge_df = pd.concat([assignment_df, taxi_df],axis=1)
merge_df

,timestamp,active,rainfall,relative-humidity,air-temperature,taxi_count
9,2022-06-01 13:15:00,1,S104,S100,S100,0.0
10,2022-06-01 13:15:00,1,S104,S100,S100,0.0
11,2022-06-01 13:15:00,1,S227,S100,S100,1.0
12,2022-06-01 13:15:00,1,S227,S100,S100,0.0
13,2022-06-01 13:15:00,1,S209,S100,S100,0.0
...,...,...,...,...,...,...
255,2022-06-01 13:15:00,1,S108,S108,S108,0.0
265,2022-06-01 13:15:00,1,S115,S121,S121,0.0
266,2022-06-01 13:15:00,1,S115,S121,S121,0.0
267,2022-06-01 13:15:00,1,S115,S121,S121,0.0


## 4. Writing to BigQuery

In [47]:
table_id='assignment-station'
assignment_df.to_gbq('.'.join([dataset_id,table_id]),project,if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 6887.20it/s]


In [52]:

taxi_table_id='assignment-taxi'
taxi_df.to_gbq('.'.join([dataset_id,taxi_table_id]),project,if_exists='append')

100%|██████████| 1/1 [00:00<00:00, 11066.77it/s]


## Feature engineering

In [ ]:
merge_df['timestamp'] = merge_df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
merge_df['hour'] = merge_df['timestamp'].apply(lambda x: x.hour)
merge_df['month'] = merge_df['timestamp'].apply(lambda x: x.month)
merge_df['day'] = merge_df['timestamp'].apply(lambda x: x.weekday())
merge_df['minute'] = merge_df['timestamp'].apply(lambda x: x.minute)

merge_df['time_30'] = merge_df['timestamp'].apply(lambda x: x + timedelta(hours=0.5))
merge_df['time_60'] = merge_df['timestamp'].apply(lambda x: x + timedelta(hours=1))

In [ ]:
## To get y_30 and y_60 targets
# for each timestamp, get 30min later
merge_df['y_30'] = np.nan
merge_df['y_60'] = np.nan

for i in tqdm(range(len(merge_df))): # for each row
    ts = merge_df.iloc[i]['time_30']
    gridnum = merge_df.iloc[i]['grid_num']
    
    merge_df.iloc[i, merge_df.columns.get_loc('y_30')] = merge_df[(merge_df['grid_num'] == gridnum) & 
                                                                       (merge_df['timestamp'] == ts)].reset_index()['taxi_count'][0]
    

for i in tqdm(range(len(merge_df))): # for each row
    ts = merge_df.iloc[i]['time_60']
    gridnum = merge_df.iloc[i]['grid_num']
    
    merge_df.iloc[i, merge_df.columns.get_loc('y_60')] = merge_df[(merge_df['grid_num'] == gridnum) & 
                                                                       (merge_df['timestamp'] == ts)].reset_index()['taxi_count'][0]
    

In [5]:
def get_training_data(start_date,end_date,project_id='ml-eng-cs611-group-project',dataset_id='taxi_dataset_views'):
    '''Query NEA BigQuery for metadata
    Args:
        start_date:         Starting date to load data
        end_date:           Ending date to load data
        project_id:         Google Cloud project_id
        dataset_id:         Google Cloud dataset_id
    Returns:
        pandas.DataFrame containing metadata for selected measure
    '''
    
    sql = f"""
    SELECT *
    FROM `{dataset_id}.final-dataframe`
    WHERE EXTRACT(DATE FROM timestamp) BETWEEN '{start_date}' AND '{end_date}'
    """

    return pd.read_gbq(sql, project_id=project_id)

df = get_training_data('2022-05-28','2022-05-30')
df

In [6]:
df = get_training_data('2022-05-28','2022-05-30')
df

,timestamp,grid_num,taxi_count,rainfall,air_temperature,relative_humidity
0,2022-05-28 00:00:00+00:00,9,0.0,0.0,27.9,NaN
1,2022-05-28 00:00:00+00:00,10,0.0,0.0,27.9,NaN
2,2022-05-28 00:00:00+00:00,11,4.0,0.0,27.9,NaN
3,2022-05-28 00:00:00+00:00,12,1.0,0.0,27.9,NaN
4,2022-05-28 00:00:00+00:00,13,0.0,0.0,27.9,NaN
...,...,...,...,...,...,...
52123,2022-05-30 23:45:00+00:00,255,0.0,NaN,NaN,NaN
52124,2022-05-30 23:45:00+00:00,265,0.0,NaN,NaN,NaN
52125,2022-05-30 23:45:00+00:00,266,0.0,NaN,NaN,NaN
52126,2022-05-30 23:45:00+00:00,267,0.0,NaN,NaN,NaN
